# Libraries

In [44]:
import pandas as pd
import numpy as np
import os
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import gensim
import spacy
from stop_words import get_stop_words
# from gensim.models import Word2Vec
from gensim.models import Word2Vec

# Data

In [2]:
data_dir='./bbc/'
print os.listdir(data_dir)
stop_= stopwords.words('english')
en_stop = get_stop_words('en')

['.DS_Store', 'entertainment', 'business', 'sport', 'politics', 'README.TXT', 'tech']


In [21]:
sports= data_dir+'sport/'
sports_=os.listdir(sports)
politics= data_dir+'politics/'
politics_= os.listdir(politics)
print len(politics_)
print len(sports_)

417
511


# Cleaning Data

In [4]:
with open(sports+sports_[0]) as f:
    file_= f.readlines()
file_
        

['Fuming Robinson blasts officials\n',
 '\n',
 'England coach Andy Robinson insisted he was "livid" after his side were denied two tries in Sunday\'s 19-13 Six Nations loss to Ireland in Dublin.\n',
 '\n',
 'Mark Cueto\'s first-half effort was ruled out for offside before the referee spurned TV replays when England crashed over in the dying minutes. "[I\'m] absolutely spitting. I\'m livid. There\'s two tries we\'ve been cost," Robinson told BBC Sport. "We\'ve got to go back to technology. I don\'t know why we didn\'t." South African referee Jonathan Kaplan ruled that Cueto was ahead of Charlie Hodgson when the fly-half hoisted his cross-field kick for the Sale wing to gather.\n',
 '\n',
 'Kaplan then declined the chance to consult the fourth official when Josh Lewsey took the ball over the Irish line under a pile of bodies for what could have been the game-winning try. "I think Mark Cueto scored a perfectly legal try and I think he should have gone to the video referee on Josh Lewsey,"

In [28]:
def cleaning_text(documents,directory_):
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    cleaned_text=[]
    for i in xrange(len(documents)):
        with open(directory_+documents[i]) as f:
            file_= f.readlines()
        for j in xrange(len(file_)):
            stripped_= file_[j].rstrip()
            lower_= stripped_.lower()
            lower_= lower_.translate(None, string.punctuation)
            try:
                tokenize_= word_tokenize(lower_)
            except UnicodeDecodeError:
                tokenize_="NA"
            digit_taken_out=[]
            for w1 in tokenize_:
                if w1.isdigit():
                    pass
                else:
                    digit_taken_out.append(w1)
                
            temp_list=[]
            for w in digit_taken_out:
                if w not in en_stop:
                    temp_list.append(w)
            cleaned_text.append(temp_list)
    print len(cleaned_text)
    print cleaned_text[0]
    return cleaned_text
cleaned_text= cleaning_text(sports_,sports)
cleaned_text_pol=cleaning_text(politics_,politics)

/Users/vprasad/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


5141
['fuming', 'robinson', 'blasts', 'officials']
4928
['baron', 'kinnock', 'makes', 'lords', 'debut']


In [29]:
cleaned_text = [x for x in cleaned_text if x != []]
cleaned_text_pol = [x for x in cleaned_text_pol if x != []]
print len(cleaned_text)
print len(cleaned_text_pol)

2801
2677


In [30]:
cleaned_text_all= cleaned_text+cleaned_text_pol

# Model

In [31]:
min_count = 5
size = 100
window = 4

#sentences = docs;
#docs = doc_topic_dict[9]
model = Word2Vec(cleaned_text_all, min_count=min_count, size=size, window=window , hs =1 ,)
#sentences = docs
# train model
#model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['sentence'])
# save model
model.save('model_survey.bin')
# load model
new_model = Word2Vec.load('model_survey.bin')


Word2Vec(vocab=5762, size=100, alpha=0.025)
['foul', 'four', 'hanging', 'cyprus', 'deport', 'sevens', 'increase', 'eligible', 'fergusons', 'lord', 'ioannidis', 'regional', 'holyrood', 'bringing', 'prize', 'andre', 'wooden', 'wednesday', 'hague', 'succession', 'croatian', 'tired', 'miller', 'mcclaren', 'second', 'attended', 'maloney', 'errors', 'increasing', 'shocks', 'specialist', 'hero', 'reporter', 'reported', 'china', 'substance', 'k', 'fouryear', 'reports', 'controversy', 'military', 'criticism', 'golden', 'divide', 'criticise', 'explained', 'replace', 'brought', 'unit', 'opponents', 'agassi', 'cheating', 'spoke', 'junes', 'valbon', 'telegraph', 'passport', 'strike', 'relay', 'successful', 'brings', 'hurt', 'hole', 'hold', 'circumstances', 'locked', 'cymrus', 'example', 'grimes', 'household', 'dragons', 'battled', 'want', 'absolute', 'complaining', 'travel', 'feature', 'hot', 'robinsons', 'wrong', 'sentencing', 'righthand', 'types', 'effective', 'helsinki', 'wins', 'sylvain', 'keep

In [32]:
cleaned_text_test_sports=cleaned_text[:15]
cleaned_text_test_pol= cleaned_text_pol[:15]

In [33]:
final_=[]
for i in xrange(len(cleaned_text_test_sports)):
    cleaned_text_0= cleaned_text_test_sports[i]
    for j in xrange(len(cleaned_text_test_pol)):
        cleaned_text_1= cleaned_text_test_pol[j]
        
        if i==j:
            pass
        else:
            sim_=[]
            for words in cleaned_text_0:
                for words1 in cleaned_text_1:
                    try:
                        sim_.append(model.wv.similarity(words,words1))
                    except KeyError:
                        pass
            final_.append((np.median(sim_),i,j))
            
            

In [41]:
print final_[:10]

[(0.49882524638943093, 0, 1), (0.55456138761290186, 0, 2), (0.55256259028501797, 0, 3), (0.32269257201731139, 0, 4), (0.46903553061274017, 0, 5), (0.52339254289011716, 0, 6), (0.46458535895485076, 0, 7), (0.50142885758503208, 0, 8), (0.5050402311400114, 0, 9), (0.45124190356575822, 0, 10)]
